In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from bokeh.tile_providers import get_provider, ESRI_IMAGERY, OSM
from bokeh.plotting import figure, show
from bokeh.io import curdoc, show, output_notebook
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Select, HoverTool
from bokeh.palettes import Spectral3
from bokeh.themes import Theme

In [ ]:
output_notebook()

In [ ]:
def bkapp(doc):
    #get data
    abridged_url = "https://pomber.github.io/covid19/timeseries.json"
    df = pd.read_json(abridged_url)

    #create datetime index
    dates = pd.to_datetime([(x['date']) for x in df['France']])

    #create global dataframe
    df_global = pd.Series(index=df.columns,dtype='object')

    for country in df.columns:
        confirmed = [x['confirmed'] for x in df[country]]
        deaths = [x['deaths'] for x in df[country]]
        recovered = [x['recovered'] for x in df[country]]
        confirmed = pd.Series(confirmed, index=dates)
        deaths = pd.Series(deaths, index=dates)
        recovered = pd.Series(recovered, index=dates)
        df_global[country] = pd.DataFrame({'confirmed': confirmed, 'recovered': recovered, 'deaths': deaths})

    #create empty DataFrame for clearing purpose
    empty_df = pd.DataFrame()
    blank = pd.Series(index=dates, dtype='object')
    df_global['BLANK'] = pd.DataFrame({'confirmed': blank, 'recovered': blank, 'deaths': blank})

    #set source to empty DataFrame with ColumnDataSource model
    source = ColumnDataSource(df_global['BLANK'])

    #create and draw figure p
    p = figure(x_axis_type='datetime')
    p.line(x='index', y='confirmed', line_color=None, source=ColumnDataSource(df_global['BLANK']), legend_label='Confirmed')
    p.line(x='index', y='recovered', line_color=None, source=ColumnDataSource(df_global['BLANK']), legend_label='Recovered')
    p.line(x='index', y='deaths', line_color=None, source=ColumnDataSource(df_global['BLANK']), legend_label='Deaths')
    p.yaxis.axis_label = 'COVID19 Cases'
    p.legend.location = "top_left"

    #create and draw figure p2 on worldmap
    tile_provider = get_provider(OSM)
    p2 = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000), x_axis_type="mercator", y_axis_type="mercator")
    p2.add_tile(tile_provider)

    #functions that converts lon/lat to MercatorX/Y
    def mercX(lon):
        k = 6378137
        return lon * (k * np.pi/180.0)

    def mercY(lat):
        k = 6378137
        return np.log(np.tan((90 + lat) * np.pi/360.0)) * k

    #calculate max deaths from df_global
    death_list = list()
    for country in df_global.index:
        death_list.append(df_global[country]['deaths'].max())
    max_deaths = max(death_list)

    #create Dataframe with (MercatorX/Y,deaths,deaths_circle)
    df_coordinates = pd.read_csv('/home/jericho/CODE/countries_coordinates.csv')
    df_coordinates.set_index('name', inplace=True)
    for country in df_coordinates.index:
        try:
            df_coordinates.loc[country,'MercatorX'] = int(mercX(df_coordinates.loc[country,'longitude']))
        except ValueError:
            df_coordinates.loc[country,'MercatorX'] = 0
        try:
            df_coordinates.loc[country,'MercatorY'] = int(mercX(df_coordinates.loc[country,'latitude']))
        except ValueError:
            df_coordinates.loc[country,'MercatorY'] = 0
        try:
            df_coordinates.loc[country,'deaths'] = df_global[country]['deaths'].max()
        except ValueError:
            df_coordinates.loc[country,'deaths'] = 0
        except KeyError:
            pass
        try:
            df_coordinates.loc[country,'deaths_circle'] = (df_global[country]['deaths'].max()*200)/max_deaths
        except KeyError:
            pass

    #draw circles by country's lat & lon
    for country in df_coordinates.index:
        try:
            p2.circle(x='MercatorX', 
                    y='MercatorY',
                    size='deaths_circle',
                    fill_color="dodgerblue", line_color="dodgerblue",
                    fill_alpha=0.01,
                    source=ColumnDataSource(df_coordinates)
                    )
        except ValueError:
            p2.circle(x=0,y=0)
        except KeyError:
            p2.circle(x=0,y=0)

    #update callback triggered by Select model
    def update_country(attr, old, new):

        doc.clear()

        global p
        global source

        country = select_country.value
        source = ColumnDataSource(df_global[country])

        p = figure(x_axis_type='datetime')
        p.line(x='index', y='confirmed', line_width=2, source=source, legend_label='Confirmed')
        p.line(x='index', y='recovered', line_width=2, source=source, color=Spectral3[1], legend_label='Recovered')
        p.line(x='index', y='deaths', line_width=2, source=source, color=Spectral3[2], legend_label='Deaths')
        p.yaxis.axis_label = country +' COVID19 Cases'
        p.legend.location = "top_left"
        p.add_tools(p_hover)

        doc.add_root(row(column(p,p2),select_country))

    #create Select model and assign callback
    select_country = Select(title="Country",  options=list(df.columns), value = '')
    select_country.on_change('value', update_country)

    #create HoverTool and add it to figure p
    p_hover = HoverTool()
    p_hover.tooltips = [('Confirmed', '@confirmed'),('Recovered', '@recovered'),('Deaths', '@deaths')]
    p.add_tools(p_hover)

    #create HoverTool and add it to figure p2
    p2_hover = HoverTool()
    p2_hover.tooltips = [('Country', '@name'),('Deaths', '@deaths')]
    p2.add_tools(p2_hover)

    #add models (row) to current document
    doc.add_root(row(column(p,p2),select_country))

In [ ]:
show(bkapp) # notebook_url="http://localhost:8888"